In [19]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [20]:
from pydantic import BaseModel
from typing import List


class Words(BaseModel):
    word: str
    pronunciation: str
    meaning_eng: str
    meaning_native: str

class WordList(BaseModel):
    words: List[Words]

In [21]:
from crewai import Crew, Agent, Task


word_finder = Agent(
    role="Searching the meaning of words",
    goal="Find the meaning of given words in English and given native language.",
    backstory="""
        You are finding the meaning of English words in dictionaries for people studying English.
        These are essential elements that you should find.
        - pronunciation : 
            The pronunciation of the words
        - meaning(English) : 
            The meaning of given words in English dictionary
            Each meaning should be provided with its grammatical category information(a./ad./v./n....).
        - meaning(native) :
            What the words mean in native language
            (in other words, how the words can translate in native language)

        e.g.
            - word : succinct
            - pronunciation : /səkˈsɪŋkt/
            - meaning(English) : a. (especially of something written or spoken) briefly and clearly expressed
            - meaing(native) : 간결한

        The words will be given as a comma-separated list.
    """,
    verbose=True,
    allow_delegation=False,
)

In [22]:
searching_task = Task(
    description="Find the meaning of given words in English and {native_lang} by searching dictionaries: {word_list}",
    agent=word_finder,
    expected_output="A List of Words Object",
    output_json=WordList,
)

In [23]:
crew = Crew(
    tasks=[
        searching_task,
    ],
    agents=[
        word_finder,
    ],
    verbose=2,
)

2025-08-31 10:38:05,041 - 140039829528704 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [24]:
result = crew.kickoff(
    inputs=dict(
        native_lang="Korean",
        word_list="unnerve",
    )
)

 [DEBUG]: == Working Agent: Searching the meaning of words
 [INFO]: == Starting Task: Find the meaning of given words in English and Korean by searching dictionaries: unnerve


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer  
Final Answer: 
- word: unnerve  
- pronunciation: /ʌnˈɜːrv/  
- meaning(English): v. to make someone feel less confident and slightly frightened  
- meaning(native): 불안하게 하다

> Finished chain.
 [DEBUG]: == [Searching the meaning of words] Task output: {
  "words": [
    {
      "word": "unnerve",
      "pronunciation": "/ʌnˈɜːrv/",
      "meaning_eng": "v. to make someone feel less confident and slightly frightened",
      "meaning_native": "불안하게 하다"
    }
  ]
}




In [25]:
import json

crewai_response = json.loads(result)
words = crewai_response['words']

words

[{'word': 'unnerve',
  'pronunciation': '/ʌnˈɜːrv/',
  'meaning_eng': 'v. to make someone feel less confident and slightly frightened',
  'meaning_native': '불안하게 하다'}]

In [9]:
from googlesheets_utils import GooglesheetUtils

googlesheet = GooglesheetUtils(spreadsheet_id='1hFNuCdmySJodQM5qsR5FJ6pkPLQc5DbXwP7h74pwTs8')

values = [[word['word'], word['meaning_eng'], word['meaning_native']] for word in words]
googlesheet.append_data('Behave!B4', values)



3 cells appended.


{'spreadsheetId': '1hFNuCdmySJodQM5qsR5FJ6pkPLQc5DbXwP7h74pwTs8',
 'tableRange': 'Behave!B4:E308',
 'updates': {'spreadsheetId': '1hFNuCdmySJodQM5qsR5FJ6pkPLQc5DbXwP7h74pwTs8',
  'updatedRange': 'Behave!B309:D309',
  'updatedRows': 1,
  'updatedColumns': 3,
  'updatedCells': 3}}

In [1]:
from googlesheets_utils import GooglesheetUtils

googlesheet = GooglesheetUtils(spreadsheet_id='1hFNuCdmySJodQM5qsR5FJ6pkPLQc5DbXwP7h74pwTs8')

googlesheet.get_columns('Behave!B5:B')

1 rows retrieved


[['ulcer',
  'recipient',
  'gland',
  'altruism',
  'sprawling',
  'leaven',
  'romp',
  'primatology',
  'hands-down',
  'magisterial',
  'synthesis',
  'magnum',
  'opus',
  'canon',
  'kindle',
  'electrify',
  'compulsion',
  'culmination',
  'on par with',
  'fabulist',
  'turf',
  'terminal',
  'erudite',
  'illuminate',
  'allusion',
  'exquisite',
  'prose',
  'encyclopedic',
  'encompass',
  'exemplary',
  'weave',
  'breadth',
  'crib',
  'alleviate',
  'endocrinology',
  'abbreviation',
  'glossary',
  'go whole hog',
  'lunge',
  'cyanide',
  'snarl',
  '*grapple',
  'viscera',
  'sever',
  'respirator',
  'fester',
  'pustulate',
  'shriek',
  'agony',
  'immerse (oneself in)',
  'clench',
  '*pertinent',
  'snicker at',
  'preach',
  'soapbox',
  'anthrax',
  'rape',
  'massacre',
  'menace',
  'eradicate',
  'schizophrenia',
  'creaky',
  'rally',
  'cerebral',
  'theology',
  'hideous',
  'impulse',
  'affiliation',
  'reconciliation',
  'egghead',
  'scare the crap ou

In [2]:
from translator_crew import TranslatorCrew

crew = TranslatorCrew(native_lang="Korean")

crew.translate("making you left pinkie bend")

 [DEBUG]: == Working Agent: Translator
 [INFO]: == Starting Task: Translate given English sentence or phrase in Korean: making you left pinkie bend


> Entering new CrewAgentExecutor chain...
I need to translate the phrase "making your left pinkie bend" into Korean. 

Final Answer: 
"making your left pinkie bend" in Korean is "왼쪽 새끼손가락을 굽히게 만들기"

> Finished chain.
 [DEBUG]: == [Translator] Task output: {
  "text": "making your left pinkie bend",
  "translation": "왼쪽 새끼손가락을 굽히게 만들기"
}




{'text': 'making your left pinkie bend', 'translation': '왼쪽 새끼손가락을 굽히게 만들기'}